# Storytelling Twitter with POSLDA

**Author: Yasir Abdurrahman**

# Background Information
**Twitter** merupakan salah satu sosial media yang populer digunakan masyarakat Indonesia. Terbatasnya jumlah karakter *tweet* sebanyak 144 karakter, tidak menjadikan nilai informasi sebuah *tweet* berkurang. Hal ini dibuktikan dengan masuknya *trending topic* tagar **#PilkadaDKI** saat pemilihan gubernur bulan April 2017 kemarin seperti pada berita [CNN](https://www.cnnindonesia.com/teknologi/20170419143713-192-208647/tagar-quick-count-dan-pilkada-dki-kuasai-twitter/). Menilik ke belakang tahun 2014, saat Pemilihan Umum Legislatif (Pemilu) taggar **#IndonesiaElectionDay** juga masuk sebagai *trending topic* pada berita [Liputan6](http://tekno.liputan6.com/read/2034443/hashtag-bertema-pemilu-dominasi-trending-topic-twitter). 

Sebuah informasi sangatlah bernilai bagi jurnalis dalam membuat sebuah berita. Pengumpulan informasi dari Twitter masih terbatas berdasarkan *trending topic*, padahal masih banyak informasi yang dapat dimanfaatkan. Pengumpulan *tweet* berdasarkan radius lokasi, kemudian dikelompokkan berdasarkan topik tertentu, dan selanjutnya disusun menjadi sebuah *storytelling* dari kumpulan *tweet* yang memiliki topik yang sama akan menjadi bahan baru bagi para jurnalis.

# Questions for Investigation
1. Bagaimana kesesuaian hasil dalam bentuk *storytelling* pada topik yang sama?
2. Topik apakah yang paling sering dibicarakan di Twitter?

# Dataset
Dataset yang digunakan berupa 1000 *tweets* hasil *crawling* sendiri. Berikut adalah struktur dataset yang digunakan:
```
id_user      : id user
username     : username pengguna twitter
created_at   : tanggal dan waktu dibuatnya tweet
latitude     : latitude
longitude    : longitude
text         : tweet
```
Dataset ditempatkan pada file dengan format *.csv*

# Crawling
#### Proses crawling
1. Menggunakan library dari python yaitu twitter, dilakukan crawling berdasarkan radius dari koordinat latitude dan longtitude
2. Data crawling meliputi atribut <b>id_user, username, created_at, latitude, longitude, text</b>
3. Hasil crawling disimpan pada file bertipe .csv

In [ ]:
import tweepy
import time
import sys
import csv

latitude = -7.059035     # geographical centre of search
longitude = 110.443972   # geographical centre of search
max_range = 10            # search range in kilometres
outfile = "tweets14.csv"

consumer_key = 'w6LdCwZHkdrIaNgxvuhZziSYe'
consumer_secret = 'Y4guQcTeCwpXJ8UhYyYzCbazxTgHZqYLpN0maNJThXXCEfrMwz'
access_token = '477563933-CIawAP0XgC8tjjXcRYmKaf4p0w2OqFAG4duYCqsL'
access_secret = 'iZIjf2p4QaNyRE7wcibb1vNpIK2JntSRfrruoYRBMs6AS'

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [ ]:
csvfile = open(outfile, "w", newline='', encoding='utf-8')
csvwriter = csv.writer(csvfile)

row = [ "id_user", "username", "created_at", "latitude", "longitude", "text" ]
csvwriter.writerow(row)

In [ ]:
result_count = 0
start_time = time.clock()

while result_count < 10000:
    c = tweepy.Cursor(api.search,
                      q="*",
                      count=10000,
                      geocode = "%f,%f,%dkm" % (latitude, longitude, max_range)).items(10000)
    while True:
        try:
            tweet = c.next()
            if tweet.geo:
                csvwriter.writerow([tweet.user.id, tweet.user.screen_name, tweet.created_at, 
                                    tweet.geo['coordinates'][0], tweet.geo['coordinates'][1], tweet.text])
                result_count += 1
                print ("got %d results" % result_count)
            else:
                csvwriter.writerow([tweet.user.id, tweet.user.screen_name, tweet.created_at, 
                                    None, None, tweet.text])
                result_count += 1
                print ("got %d results" % result_count)

        except tweepy.TweepError:
            print("sleeping")
            time.sleep(15 * 60)
            continue
        except StopIteration:
            break
csvfile.close()
print(time.clock() - start_time, "seconds")

# Preprocessing
Hal yang dilakukan:
1. Common Preprocessing
    1. Removing symbols, ASCII strings, punctuations.
    2. Tokenization
    3. Case folding, convert into lowercase
    4. Repeated dot (sedih... -> sedih.)
    5. Repeated character ('hehe :)))' -> 'hehe :)')
    6. Remove elipsis (lanjut baca... -> lanjut baca)
    7. Repeated word that has meaning ('malam malam' -> 'malam-malam')
    8. Remove newline
2. Specific Preprocessing
    1. Special symbols on Twitter, removing hashtag and mention
    2. Remove all emoticons
    3. Remove URL
    4. Spell checker using Jaro Winkler

In [14]:
import pandas as pd

In [15]:
df_tweets = pd.read_csv('tweets12.csv')
df_tweets.shape

(500, 6)

In [16]:
df_tweets.head()

,id_user,username,created_at,latitude,longitude,text
0,2313451298,dikyock,2018-01-24 04:49:23,NaN,NaN,@rossonerifreak Masih kurang \nButuh winger ta...
1,245873655,phee_nophee,2018-01-24 04:49:21,NaN,NaN,Apa yang Akan Berubah Untukmu Tahun Ini? https...
2,955268887406264321,FahrulAditya16,2018-01-24 04:49:21,NaN,NaN,RT @Pesona_STW: Support by (˛`̯´̯)-☞\n@Copasaj...
3,791650560961241088,isarndr88,2018-01-24 04:49:10,NaN,NaN,Apakah IPK penting?
4,2395660873,Lukman_Prayogoo,2018-01-24 04:49:09,NaN,NaN,https://t.co/YLyXSx3EVh


In [17]:
from modulenorm.normalize import Normalize
from modulenorm.tokenize import Tokenize

In [18]:
idx = 0
df_tweets['normalize'] = None
result = []
for row in df_tweets['text']:
    # normalize
    norm = Normalize()
    text_norm = norm.remove_ascii(row)
    text_norm = norm.lower_text(text_norm)
    text_norm = norm.repeat_char_modify(text_norm)
    text_norm = norm.remove_elipsis(text_norm)
    text_norm = norm.remove_newline(text_norm)
    text_norm = norm.remove_url(text_norm)
#     text_norm = norm.remove_emoticons(text_norm)
    text_norm = norm.remove_hashtags_mentions(text_norm)
    
    tok = Tokenize()
    text_norm = tok.WordTokenize(text_norm)
    result.append(text_norm)
    
    text_norm = ' '.join(text_norm)
#     df_tweets['normalize'][idx] = text_norm
    idx += 1

In [19]:
for kalimat in result:
    print(kalimat)

['masih', 'kurang', 'butuh', 'winger', 'tajam', 'skil', 'hebat', 'untuk', 'menusuk', 'pertahanan', 'lawan', 'butuh', 'stiker', 'tajam', 'ump']
['apa', 'yang', 'akan', 'berubah', 'untukmu', 'tahun', 'ini']
['rt', 'stw', 'support', 'by']
['apakah', 'ipk', 'penting']
['']
['rt', '1', 'stand', 'boom', 'pearl', '650', '000', '2', 'stand', 'boom', 'tama', '650', '000', '3', 'meinl', 'classics', 'custom', 'extreme', 'metal', 'china', '16', '1', '200', '000', '4']
['o8']
['rt', 'tindakannya', 'misal', 'bagi', 'petani', 'kendeng', 'dan', 'kulonprogo', 'retorika', 'tidaklah', 'cukup']
['ini', 'jujur', 'mbak', 'e', 'jujur', 'kacang', 'hejo']
['current', 'weather', 'in', 'semarang', 'broken', 'clouds', '27c', '80', 'humidity', 'wind', '4kmh', 'pressure', '970mb']
['']
['wah', 'teknik', 'nih']
['oke', 'pulang', 'lebaran']
['o7']
['rt', 'gempa', 'jangan', 'dibecandain']
['astagfirullah', 'ini', 'clickbait', 'yg', 'membangun']
['yes', 'some', 'chapters', 'have', 'no', 'discretion', 'anymore', 'they',

['rt', 'umin', 'exol', 'bestfanarmy', 'iheartawards']
['rt', 'now', 'everybody', 'know', 'that', 'inside', 'babang', 'selebgram']
['opini', 'aku', 'pilih', 'ahok', 'kok']
['rt', 'latihan', 'psis', 'semarang', 'untuk', 'menjaga', 'kondisi', 'dan', 'kebugaran', 'pemain', 'selasa', '23', '1', 'psis']
['serem', 'banget', 'ded', 'laporin', 'ke', 'pak', 'rt', 'cobak']
['newprofilepic']
['sedikit', 'orang', 'kaya', 'yang', 'memiliki', 'harta', 'kebanyakan', 'harta', 'yang', 'memiliki', 'mereka', 'robert', 'g', 'ingersoll', 'simpelize']
['sama', 'bosmu', 'mesti', 'ditenani', 'tapi', 'kamu', 'yang', 'akan', 'lelah']
['sabar']
['chaeyeon', 'pict']
['hayo', 'mas']
['rt', 'toetik']
['chaeyeon', 'gifs']
['rt', 'giat', 'transit', 'lps', 'rw', '04', 'kel', 'meruya', 'utara', 'rabu', '24', '01', '2018']
['bona', 'pict']
['rt', 'giat', 'rutin', 'germor', 'satpel', 'kembangan', 'jl', 'lingkar', 'dalam', 'cni', 'kembangan', 'selatan', 'rabu', '24', '01', '2018']
['genki', 'udah', 'terlihat', 'mulai', 'ik

In [20]:
from pyjarowinkler import distance
import operator

In [ ]:
fin = open("kamus.txt", encoding='utf8')
kamus = fin.readlines()
kamus = [item.rstrip('\n') for item in kamus]
fin.close()

i = 0
lstkalimatbr = []
lstdokumenbr = []
for kalimat in result:
    lstkalimatbr = []
    for kata in kalimat:
        per_kata = {}
        for kms in kamus:
            result = distance.get_jaro_distance(repr(kata), repr(kms), winkler=True, scaling=0.1)
            per_kata.update({i: (result, kata, kamus)})
            i = i + 1
        maxi = max(per_kata.items(), key=operator.itemgetter(1))
        lstkalimatbr.append(maxi[1][2])
    lstdokumenbr.append(lstkalimatbr)

In [ ]:
df_tweets['normalize'].to_csv('nnn.csv', header=None, index=False)

In [ ]:
idx = 0
result = []
for row in df['text']:
    
    # normalize
    usenorm = normalize()
    text_norm = usenorm.remove_ascii(row) # normalisasi enter, 1 revw 1 baris
    text_norm = usenorm.enterNormalize(text_norm) # normalisasi enter, 1 revw 1 baris
    text_norm = usenorm.lowerNormalize(text_norm) # normalisasi huruf besar ke kecil
    text_norm = usenorm.repeatcharNormalize(text_norm) # normalisasi titik yang berulang
    text_norm = usenorm.linkNormalize(text_norm) # normalisasi link dalam text
    text_norm = usenorm.spacecharNormalize(text_norm) # normalisasi spasi karakter
    text_norm = usenorm.ellipsisNormalize(text_norm) # normalisasi elepsis (…)
    
    # tokenize
    tok = tokenize() 
    text_norm = tok.WordTokenize(text_norm) # pisah tiap kata pada kalimat
    result.append(text_norm)
    
    
#     text_norm = usenorm.spellNormalize(text_norm) # cek spell dari kata perkata
#     text_norm = usenorm.wordcNormalize(text_norm,2) # menyambung kata (malam-malam) (param: textlist, jmlh_loop)
    # text_norm = usenorm.stemmingNormalize(text_norm,'word') # mengubah ke bentuk kata dasar (text, type_data)

    text_norm = ' '.join(text_norm) # menggabung kalimat tokenize dengan separate spasi

#     text_norm = usenorm.emoticonNormalize(text_norm) # menggabung struktur emoticon yang terpisah ([: - )] = [:-)])

    # walking2
    # konfer @ ke at untuk penunjuk tempat
    
#     output = open("output1.txt","a")
#     output.write(str(text_norm))
#     output.write('\n')
#     output.close()

#     print(no, text_norm)
#     df['text'][idx] = text_norm
    idx += 1

In [ ]:
df['text'].to_csv('ttt1.csv', header=['tweets'], index=False)

In [ ]:
texts = df['text']
texts.head()

In [ ]:
print(df['text'][4])
type(df['text'][4])

In [ ]:
def encode_ascii_replace(row):
    return row.encode('ascii', 'replace')

In [ ]:
t = df['text']
t = t.apply(encode_ascii_replace)

In [ ]:
t[4].decode('utf-8')

In [ ]:
def decode(row):
    return row.decode('utf-8')

In [ ]:
td = t.apply(decode)

In [ ]:
td.to_csv('text9.csv', header=['tweets'], index=False)

## Text Data Cleaning

In [ ]:
texts_df = pd.read_csv('text9.csv')
texts_df.head()

In [ ]:
import html

def html_parser(row):
    return html.unescape(row)

In [ ]:
tdf = texts_df['tweets']
clean1 = tdf.apply(html_parser)
clean1[22]

In [ ]:
def decoding_data(row):
    return row.encode('ascii', 'ignore').decode('utf-8')

In [ ]:
clean2 = clean1.apply(decoding_data)
clean2[53]

In [ ]:
import itertools

def standardizing_words(row):
    return ''.join(''.join(s)[:2] for _, s in itertools.groupby(row) )

In [ ]:
clean3 = clean2.apply(standardizing_words)
print(clean2[49])
clean3[49]

In [ ]:
import re

def clean_url(row):
    return re.sub(r"http\S+", "", row)

In [ ]:
clean4 = clean3.apply(clean_url)
print(clean3[0])
clean4[0]

In [ ]:
def remove_newline(row):
    return ' '.join(row.splitlines())

In [ ]:
clean5 = clean4.apply(remove_newline)
clean5[0]

In [ ]:
from string import maketrans

intab = "=><;()^{}[]\~"
outtab = ""
trantab = maketrans(intab, outtab)

def remove_unnecessary_punctuation(row):
    return row.translate(trantab)

In [ ]:
# create sample documents
doc_a = "Brokoli bagus untuk dimakan. Adikku suka makan brokoli, tetapi ibuku tidak."
doc_b = "Ibuku menghabiskan banyak waktu berkeliling melihat adikku latihan bisbol."
doc_c = "Beberapa ahli kesehatan menyarankan bahwa mengemudi dapat menyebabkan ketegangan dan tekanan darah meningkat."
doc_d = "Saya sering merasakan tekanan untuk tampil seperti saat presentasi di sekolah."
doc_e = "Profesional kesehatan mengatakan bahwa brokoli itu baik untuk kesehatan."
doc_f = "Teman saya seorang pemain bisbol yang pernah mendapatkan juara."
doc_g = "Pemain bisbol yang bernama Flash itu sangat suka memakan brokoli."
doc_h = "Sopir yang mengemudi taksi itu mendapatkan tekanan dari penumpangnya."
doc_i = "Saat bertanding, olahraga bisbol memberikan ketegangan dan meningkatkan tekanan darah para penonton."
doc_j = "Ibuku menyarankan saya untuk memakan brokoli agar tekanan darah terkontrol."

# compile sample documents into a list
docs = [doc_a, doc_b, doc_c, doc_d, doc_e, doc_f, doc_g, doc_h, doc_i, doc_j]

In [ ]:
from nltk.tokenize import RegexpTokenizer
doc_j = "Ibuku menyarankan saya untuk memakan brokoli agar tekanan darah terkontrol."
a = [doc_j]

tokenizer = RegexpTokenizer(r'\w+|\$[\d\.]+')
print(tokenizer.tokenize(doc_j))

In [ ]:
result = []
for doc in docs:
    print(doc)
    tokenizer = RegexpTokenizer(r'\w+|\$[\d\.]+')
    result.append(tokenizer.tokenize(doc))

for i in result:
    print(i)

In [ ]:
from modulenorm.tokenize import Tokenize

In [ ]:
tok = Tokenize()